# EXECUTE ALL CELLS BY HITTING CTRL + ENTER TOGETHER

In [27]:
import time
import requests
import json
import openpyxl

In [28]:
import pandas as pd

In [29]:
def withinTimeInterval(start,end, timestamp):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    timestamp = pd.to_datetime(timestamp)
    if (timestamp >= start) and (timestamp <= end):
        return True
    else:
        return False

In [30]:
def JFKToAPoint(lat1,long1):
    reqString = "http://localhost:8989/route?point=40.646016%2C-73.778438&point="+str(lat1)+"%2C"+str(long1)+"&vehicle=car"
    
    return requests.get(reqString)


In [31]:
def JFKToAPointDist(lat,long):
    response = JFKToAPoint(lat,long)
    if 'paths' in response.json().keys():
        return response.json()['paths'][0]['distance']
    return 0


# 1) Change Excel File Name to: "Delay1_2.xlsx" or "...1_3.xlsx" or "....1_5.xlsx" to get data for the desired delay factor

In [ ]:
readData = pd.ExcelFile("Delay1_3.xlsx")
JFKStarts = readData.parse("Sheet1")
#JFKStarts.head()

In [34]:
def createPools(startDate,noOfPools,poolWindow): #('5/1/13',10,2min)
    res = pd.date_range(start=startDate, periods = noOfPools, freq=poolWindow)
    print(res)
    poolList = []
    i=0
    print("Creating Pools: ")
    for i in range(noOfPools-1):
        print("Creating Pool:")
        print(res[i],res[i+1]-pd.Timedelta('1min'))
        pool = JFKStarts[JFKStarts.apply(lambda x: withinTimeInterval(res[i],res[i+1]-pd.Timedelta('1min'),x[' pickup_datetime']), axis =1)]
        poolList.append(pool)
   
    return poolList
       
    

In [58]:
def mainFunction(startDate,noOfPools,poolWindow):
    poolList = createPools(startDate,noOfPools+1,poolWindow)
    poolData = []
    for pool in poolList:
        poolData.append(createPoolData(pool))
       
    rowValue = 1
    for pool in poolData:
        print("*****************************")
        start_time = time.clock()
        processPool(pool, rowValue)
        end_time = time.clock() - start_time
        print("Computation Time (Seconds): "+ str(end_time))
        #print(end_time)
        
        sheet1.write(rowValue,12,end_time)
        rowValue +=1
    
    book1.close()

In [59]:
def mainFunction2(startDate,noOfPools,poolWindow):
    poolList = createPools(startDate,noOfPools+1,poolWindow)
    poolData = []
    for pool in poolList:
        poolData.append(createPoolData(pool))
       
    rowValue = 1
    for pool in poolData:
        print("*****************************")
        start_time = time.clock()
        ridesharingWithKas2(pool, rowValue)
        end_time = time.clock() - start_time
        print("Computation Time (Seconds): "+ str(end_time))
        #print(end_time)
        
        sheet2.write(rowValue,9,end_time)
        rowValue +=1
    
    book2.close()

In [37]:
def createPoolData(pool):
    PoolData = {}
    PoolDict ={}
    for index,row in pool.iterrows():
        PoolDict[index] = {'passenger_count':row[' passenger_count'],'dropoff_longitude':row[' dropoff_longitude'],'dropoff_latitude':row[' dropoff_latitude'], 'delay_time':row['delay_time'],'org_trip_dist':row[' trip_distance']}
        
    PoolData = { 'sumOfIndividualTrips':pool[' trip_distance'].sum(), 'dict':PoolDict}
    return PoolData

In [38]:
#find all possible triplets combinations for a given triplet (dictionary ranges from 1 to 6)

def findAllPossibleCombinations(tripletMatched):
    possibleTripletsCombiDict = {}
    i=0
    for combi in itertools.permutations(list(tripletMatched),3):
        i+=1
        possibleTripletsCombiDict[i] = {'triplet':combi, 'distance':0, 'time':0}
    return possibleTripletsCombiDict

In [39]:
#Obtain the request string
def getInputData(lat1, long1, lat2, long2, lat3, long3):
    reqString = "http://localhost:8989/route?point=40.646016%2C-73.778438&point="+str(lat1)+"%2C"+str(long1)+"&point="+str(lat2)+"%2C"+str(long2)+"&point="+str(lat3)+"%2C"+str(long3)+"&vehicle=car"
    return reqString

In [40]:
def getInputData2Points(lat1, long1, lat2, long2):
    reqString = "http://localhost:8989/route?point=40.646016%2C-73.778438&point="+str(lat1)+"%2C"+str(long1)+"&point="+str(lat2)+"%2C"+str(long2)+"&vehicle=car"
    return reqString
    

In [41]:
def sendRequest(triplet,PoolDict):
    
    lat1 = PoolDict[triplet[0]]['dropoff_latitude']
    long1 = PoolDict[triplet[0]]['dropoff_longitude']
    lat2 = PoolDict[triplet[1]]['dropoff_latitude']
    long2 = PoolDict[triplet[1]]['dropoff_longitude']
    lat3 = PoolDict[triplet[2]]['dropoff_latitude']
    long3 = PoolDict[triplet[2]]['dropoff_longitude']
    return requests.get(getInputData(lat1,long1,lat2,long2,lat3,long3))

In [42]:
def sendRequest2Points(pair,PoolDict):
    lat1 = PoolDict[pair[0]]['dropoff_latitude']
    long1 = PoolDict[pair[0]]['dropoff_longitude']
    lat2 = PoolDict[pair[1]]['dropoff_latitude']
    long2 = PoolDict[pair[1]]['dropoff_longitude']
    return requests.get(getInputData2Points(lat1,long1,lat2,long2))
    

In [43]:
def processAllCombinationsForATriplet(triplet, PoolDict):
    #print(triplet)
    AllCombis = findAllPossibleCombinations(triplet)
    #Find out distance, time for every combination
    for key,value in list(AllCombis.items()):
            #print('******************')
            #print(key)
            response = sendRequest(value['triplet'],PoolDict)
            if 'paths' in response.json().keys():
                distance = response.json()['paths'][0]['distance']
                time = response.json()['paths'][0]['time']
                AllCombis[key]['distance'] = distance
                AllCombis[key]['time'] = time
            else:
                #print(AllCombis)
                del AllCombis[key]
    #print([AllCombis[x]['distance'] for x in AllCombis])        
    
    if AllCombis:
        minDistance = min([AllCombis[x]['distance'] for x in AllCombis]) 
    else:
        return 0
    
    for key,value in AllCombis.items():
        if value['distance'] == minDistance:
            bestCombination = key
    
    return constraintsViolated(AllCombis[bestCombination],PoolDict)          
            

In [44]:
def processAllCombinationsForAPair(pair, PoolDict):
    AllCombis = findAllPossibleCombinationsPair(pair, PoolDict)
    #Find out distance, time for every combination
    for key,value in list(AllCombis.items()):
            response = sendRequest2Points(value['pair'], PoolDict)
            if 'paths' in response.json().keys():
                distance = response.json()['paths'][0]['distance']
                time = response.json()['paths'][0]['time']
                AllCombis[key]['distance'] = distance
                AllCombis[key]['time'] = time
            else:
                del AllCombis[key]
    #print(AllCombis)            
    if AllCombis:
        minDistance = min([AllCombis[x]['distance'] for x in AllCombis]) 
    else:
        return 0
    
    for key,value in AllCombis.items():
        if value['distance'] == minDistance:
            bestCombination = key
    
    return constraintsViolated2Points(AllCombis[bestCombination],PoolDict)          
            

In [45]:
#find all possible triplets combinations for a given triplet (dictionary ranges from 1 to 6)
def findAllPossibleCombinationsPair(pair, PoolDict):
    possiblePairsCombiDict = {}
    i=0
    for combi in itertools.permutations(list(pair),2):
        i+=1
        possiblePairsCombiDict[i] = {'pair':combi, 'distance':0, 'time':0}
    return possiblePairsCombiDict

In [46]:
def constraintsViolated2Points(pathData,PoolDict):
    points = pathData['pair']
    #print(pathData)
    #JFK to point1
    resp1 = JFKToAPoint(PoolDict[points[0]]['dropoff_latitude'],PoolDict[points[0]]['dropoff_longitude'])
    if 'paths' in resp1.json().keys():
        time1 = resp1.json()['paths'][0]['time']
        dist1 = resp1.json()['paths'][0]['distance']
    else:
        return 0

    
    if PoolDict[points[1]]['delay_time'] < pathData['time']:
        return 0
    
    
    #Distance Constraint Check
    distSaved = (PoolDict[points[0]]['org_trip_dist']+PoolDict[points[1]]['org_trip_dist']) - pathData['distance'] 
   
    if distSaved < 0:
        return 0
    
 
    return distSaved
    

In [47]:
def constraintsViolated(pathData, PoolDict):
    points = pathData['triplet']
    #print(pathData)
    #JFK to point1
    resp1 = JFKToAPoint(PoolDict[points[0]]['dropoff_latitude'],PoolDict[points[0]]['dropoff_longitude'])
    if 'paths' in resp1.json().keys():
        time1 = resp1.json()['paths'][0]['time']
        dist1 = resp1.json()['paths'][0]['distance']
    else:
        return 0

    resp2 = sendRequest2Points((points[0],points[1]),PoolDict)
    if 'paths' in resp2.json().keys():
        time2 = resp2.json()['paths'][0]['time']
        dist2 = resp2.json()['paths'][0]['distance']
    else:
        return 0
    
    #Delay Time Constraint Check
    if PoolDict[points[1]]['delay_time'] < (time1 + time2):
        return 0
    
 
    if PoolDict[points[2]]['delay_time'] < pathData['time']:
        return 0
    
    
    #Distance Constraint Check
    distSaved = (PoolDict[points[0]]['org_trip_dist']+PoolDict[points[1]]['org_trip_dist']+PoolDict[points[2]]['org_trip_dist'] ) - pathData['distance'] 
     
    if distSaved < 0:
        return 0
    
    return distSaved
    

# Processing for k=3

In [48]:
import itertools

def processPool(PoolData, rowValue):
    
    PoolDict = PoolData['dict']
    #Find All valid triplets
    validTriplets = {}
    for item in itertools.combinations(list(PoolDict.keys()),3): #AllTriplets
        total_passengers = 0
        for x in item:
            total_passengers += PoolDict[x]['passenger_count']

        if total_passengers <= 4:
            distSaved = processAllCombinationsForATriplet(item,PoolDict)
            if distSaved != 0:
                validTriplets[item] = distSaved

    individualTrips = list(PoolDict.keys()).copy()
    validTriplets1 = validTriplets.copy()
    
    #Greedily Match Valid Triplets
    matchedTrips = []

    while validTriplets1:
        #print(len(validTriplets1))
        bestTriplet = list(validTriplets1.keys())[list(validTriplets1.values()).index(max(validTriplets1.values()))]
        #Delete all tuples that contain any of the best triplets
        for item in bestTriplet:
            for key in list(validTriplets1):
                if item in key:
                    del validTriplets1[key]
            individualTrips.remove(item)

        #Append to matchedTrips
        matchedTrips.append(bestTriplet)
    #macthedTrips holds all greedily macthed tripelts
    
    #Maximum matching the remaining individual trips
    
    validPairs = {}
    for item in itertools.combinations(individualTrips,2): #AllRemainingPairs
        total_passengers = 0
        for x in item:
            total_passengers += PoolDict[x]['passenger_count']
        if total_passengers <= 4:
            distSaved = processAllCombinationsForAPair(item, PoolDict)
            if distSaved > 0:
                validPairs[item] = distSaved
    
    #Matching the remaing individual trips
    import networkx as nx
    #Forming the sharability network
    G=nx.Graph()
    G.add_nodes_from(individualTrips)
    for pair in validPairs.keys():
        G.add_edge(pair[0],pair[1],weight = validPairs[pair])
        
    
    max_matching = nx.max_weight_matching(G,maxcardinality = True)
    #Max matching provides both pairs (a,b) and (b,a); Removing one of them
    for key,value in list(max_matching.items()):
        #print("*******")
        #print(max_matching)
        if (value,key) in max_matching.items():
            del max_matching[key]
        #print(max_matching)

    #Calculate distance saved from pairs   
    distSavedFromPairs = 0
    for key in max_matching.keys():
        if (key,max_matching[key]) in validPairs.keys():
            distSavedFromPairs += validPairs[(key,max_matching[key])]
        elif (max_matching[key],key) in validPairs.keys():
            distSavedFromPairs += validPairs[(max_matching[key],key)]
    print("Distance Saved from Pairs: "+str(distSavedFromPairs/1609.34))
    sheet1.write(rowValue,0,distSavedFromPairs/1609.34)
    
    distSavedFromTriplets = 0
    for triplet in matchedTrips:
        distSavedFromTriplets+= validTriplets[triplet]
    print("Distance saved from Triplets: "+str(distSavedFromTriplets/1609.34))
    
    sheet1.write(rowValue,1,distSavedFromTriplets/1609.34)
    totalRideSharingSavings = distSavedFromTriplets + distSavedFromPairs
    print("Total distance saved from Ride-Sharing: "+ str(totalRideSharingSavings/1609.34))
    
    sheet1.write(rowValue,2,totalRideSharingSavings/1609.34)
    
    sumOfInd = PoolData['sumOfIndividualTrips']
    print("Total distance travelled without Ride-Sharing: "+str(sumOfInd/1609.34))
    
    sheet1.write(rowValue,3,sumOfInd/1609.34)
    
    print("Total distance travelled with Ride-Sharing: "+str((sumOfInd - totalRideSharingSavings)/1609.34))
    sheet1.write(rowValue,4,(sumOfInd - totalRideSharingSavings)/1609.34)
    
    pairedTrips = [item for k in max_matching for item in (k, max_matching[k])]
    numberOfLoneTrips = len([x for x in individualTrips if x not in pairedTrips])
    numberOfMergedTrips = len(matchedTrips) + len(max_matching)
    totalTripsMerged = numberOfLoneTrips + numberOfMergedTrips
    if sumOfInd !=0:
        percentDistSaved = ((sumOfInd - totalRideSharingSavings)/sumOfInd)*100
    else:
        percentDistSaved = 0
    
    print("Percentage of Original distance travelled: "+str(percentDistSaved))
    sheet1.write(rowValue,5,percentDistSaved)
    
    print("Total number of individual trips: "+str(len(PoolDict.keys())))
    sheet1.write(rowValue,6,len(PoolDict.keys()))
    
    print("Number of merged trips each of size 3: "+str(len(matchedTrips)))
    sheet1.write(rowValue,7,len(matchedTrips))
    
    print("Number of merged trips each of size 2: "+str(len(max_matching)))
    sheet1.write(rowValue,8,len(max_matching))
    
    print("Number of lone trips: "+str(numberOfLoneTrips))
    sheet1.write(rowValue,9,numberOfLoneTrips)
    
    print("Number of trips after ride-sharing: "+str(totalTripsMerged))
    sheet1.write(rowValue,10,totalTripsMerged)
    
    if len(PoolDict.keys()) != 0:
        #percentTripsSaved = ((len(PoolDict.keys()) - totalTripsMerged )/len(PoolDict.keys()))*100
        percentTripsSaved = ((totalTripsMerged )/len(PoolDict.keys()))*100
    else:
        percentTripsSaved = 0
    print("Percentage of trips saved: "+str(percentTripsSaved))
    sheet1.write(rowValue,11,percentTripsSaved)
    
    


    

# Processing for k=2

In [49]:
def ridesharingWithKas2(PoolData, rowValue):
    PoolDict = PoolData['dict']
    validPairs = {}
    individualTrips = list(PoolDict.keys()).copy()
    for item in itertools.combinations(list(PoolDict.keys()),2): #AllRemainingPairs
        total_passengers = 0
        for x in item:
            total_passengers += PoolDict[x]['passenger_count']
        if total_passengers <= 4:
            distSaved = processAllCombinationsForAPair(item, PoolDict)
            if distSaved > 0:
                validPairs[item] = distSaved
    
    #Matching the remaing individual trips
    import networkx as nx
    #Forming the sharability network
    G=nx.Graph()
    G.add_nodes_from(individualTrips)
    for pair in validPairs.keys():
        G.add_edge(pair[0],pair[1],weight = validPairs[pair])
        
    
    max_matching = nx.max_weight_matching(G,maxcardinality = True)
    #Max matching provides both pairs (a,b) and (b,a); Removing one of them
    for key,value in list(max_matching.items()):
        if (value,key) in max_matching.items():
            del max_matching[key]
        #print(max_matching)

    #Calculate distance saved from pairs   
    distSavedFromPairs = 0
    for key in max_matching.keys():
        if (key,max_matching[key]) in validPairs.keys():
            distSavedFromPairs += validPairs[(key,max_matching[key])]
        elif (max_matching[key],key) in validPairs.keys():
            distSavedFromPairs += validPairs[(max_matching[key],key)]
    print("Distance Saved from Pairs: "+str(distSavedFromPairs/1609.34))
    sheet2.write(rowValue,0,distSavedFromPairs/1609.34)
    
    
    
    sumOfInd = PoolData['sumOfIndividualTrips']
    print("Total distance travelled without Ride-Sharing: "+str(sumOfInd/1609.34))
    
    sheet2.write(rowValue,1,sumOfInd/1609.34)
    
    print("Total distance travelled with Ride-Sharing: "+str((sumOfInd - distSavedFromPairs)/1609.34))
    #sheet2.write(rowValue,2,(sumOfInd - distSavedFromPairs)/1609.34)
    
    #print(max_matching)
    pairedTrips = [item for k in max_matching for item in (k, max_matching[k])]
    loneTrips = [x for x in individualTrips if x not in pairedTrips]
    loneTripDist = 0
    for trip in loneTrips:
        loneTripDist += PoolDict[trip]['org_trip_dist']
     
    print("Lone Trip Distance: "+str(loneTripDist/1608))
    numberOfLoneTrips = len([x for x in individualTrips if x not in pairedTrips])
    totalTripsMerged = numberOfLoneTrips + len(max_matching)
    
    if sumOfInd !=0:
        percentDistSaved = ((sumOfInd - distSavedFromPairs)/sumOfInd)*100
    else:
        percentDistSaved = 0
    
    print("Percentage of original distance travelled: "+str(percentDistSaved))
    sheet2.write(rowValue,3,percentDistSaved)
    
    print("Total number of individual trips: "+str(len(PoolDict.keys())))
    sheet2.write(rowValue,4,len(PoolDict.keys()))
    
    print("Number of merged trips each of size 2: "+str(len(max_matching)))
    sheet2.write(rowValue,5,len(max_matching))
    
    print("Number of lone trips: "+str(numberOfLoneTrips))
    sheet2.write(rowValue,6,numberOfLoneTrips)
    
    print("Number of trips after ride-sharing: "+str(totalTripsMerged))
    sheet2.write(rowValue,7,totalTripsMerged)
    
    if len(PoolDict.keys()) != 0:
        percentTripsSaved = ((totalTripsMerged )/len(PoolDict.keys()))*100
    else:
        percentTripsSaved = 0
    print("Percentage of original trips when ride-sharing: "+str(percentTripsSaved))
    sheet2.write(rowValue,8,percentTripsSaved)
    

# RUN TEST FOR K = 3 BELOW

## 2) Change excel file name below

In [63]:
import xlsxwriter as xw

book1 = xw.Workbook('TEST_K3_PW3_DF13.xlsx')
sheet1 = book1.add_worksheet()


## 3) Change number of pools to process (2nd parameter)

## 4) Change pool window size in minutes (3rd parameter)

In [ ]:
%%time
#k=3
#mainFunction(start date and time, number of pools to process, poolwindow)
#time is in military time
mainFunction('5/1/13 12:00:00',5,'3min')

# RUN TEST FOR K = 2 BELOW

## 2) Change excel file name below

In [65]:
book2 = xw.Workbook('TEST_K2_PW3_DF13.xlsx')
sheet2 = book2.add_worksheet()

## 3) Change number of pools to process (2nd parameter)

## 4) Change pool window size in minutes (3rd parameter)

In [ ]:
%%time
#for k=2
#mainFunction2(start date and time, number of pools to process, poolwindow)
#time is in military time
mainFunction2('5/1/13 12:00:00',5,'3min')